In [1]:
import grpc
from concurrent import futures
import threading
import time

import broker_pb2 as pb2
import broker_pb2_grpc as grpc2

import os
import queue
from pydub import AudioSegment

import logging
logger = logging.getLogger(__name__)
logger.setLevel(logging.DEBUG)

In [2]:
port = 8061


In [3]:
def PickFile(audioid, path):
    
    sound = AudioSegment.from_wav(path)
    return audioid, sound

In [4]:
class send_fileServiceImpl(grpc2.send_fileServicer):
    
    def __init__(self, queue):
        self.queue = queue


    def Send(self, request, context):
        
        self.queue.get(block=True)
        filepath = "./audio.waw"
        audioid = 1
        
        file = pb2.AudioFile()
        file.audioid, file.audiofile = PickFile(audioid, filepath)
        
        logger.debug("Sending audio: {}".format(file.audioid))
        print("Sending audio: {}".format(file.audioid))
        
        return file

In [5]:
server = grpc.server(futures.ThreadPoolExecutor(max_workers = 10))
q = queue.Queue()
grpc2.add_send_fileServicer_to_server(send_fileServiceImpl(q), server)
print("Starting server, Listening on port:" + str(port))
server.add_insecure_port("[::]:{}".format(port))
server.start()
server.wait_for_termination()

Starting server, Listening on port:8061


KeyboardInterrupt: 